In [ ]:
!pip install audformat
!pip install opensmile
!pip install Keras-Preprocessing

In [ ]:
!wget https://zenodo.org/record/7447302/files/emodb.zip

!unzip emodb.zip

--2023-11-12 09:09:29--  https://zenodo.org/record/7447302/files/emodb.zip
Resolving zenodo.org (zenodo.org)... 188.185.10.78, 188.185.22.33, 188.185.33.206, ...
Connecting to zenodo.org (zenodo.org)|188.185.10.78|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7447302/files/emodb.zip [following]
--2023-11-12 09:09:30--  https://zenodo.org/records/7447302/files/emodb.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 39981818 (38M) [application/octet-stream]
Saving to: ‘emodb.zip’

emodb.zip           100%[===================>]  38.13M  13.3MB/s    in 2.9s    

2023-11-12 09:09:33 (13.3 MB/s) - ‘emodb.zip’ saved [39981818/39981818]

Archive:  emodb.zip
   creating: emodb/
  inflating: emodb/db.emotion.categories.train.gold_standard.pkl  
   creating: emodb/wav/
  inflating: emodb/wav/14b01Fc.wav   
  inflating: emodb/wav/14a02Ea.wav   
  inflating: emodb/wav/15a02Ta.wav   
  infla

**Loading libraries**

In [ ]:
import audformat

import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

from keras_preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import keras
from keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping

import opensmile

import pandas as pd
import matplotlib.pyplot as plt
import torch

**Loading dataset**

using the gold standard train and test data

In [ ]:

db = audformat.Database.load('emodb')

df_test = db['emotion.categories.test.gold_standard'].get()

df_train = db['emotion.categories.train.gold_standard'].get()

print(f'samples in train: {df_train.shape[0]}, and test: {df_test.shape[0]}')



samples in train: 304, and test: 231


In [ ]:
# Encode the emotion words as numbers and use this as target
target = 'label'
encoder = LabelEncoder()
encoder.fit(df_train['emotion'])
df_train[target] = encoder.transform(df_train['emotion'])
df_test[target] = encoder.transform(df_test['emotion'])

In [ ]:
df_test[target]

file
wav/12a01Fb.wav    4
wav/12a01Lb.wav    1
wav/12a01Nb.wav    5
wav/12a01Wc.wav    0
wav/12a02Ac.wav    3
                  ..
wav/16b10Lb.wav    1
wav/16b10Tb.wav    6
wav/16b10Td.wav    6
wav/16b10Wa.wav    0
wav/16b10Wb.wav    0
Name: label, Length: 231, dtype: int64

Using opensmile to extract raw features

In [ ]:


smile = opensmile.Smile(
    opensmile.FeatureSet.ComParE_2016,
    opensmile.FeatureLevel.Functionals,
    sampling_rate=16000,
    resample=True,
    num_workers=5,
    verbose=True,
)

X_train = smile.process_index(
    df_train.index,
    root=db.root,

)
X_test = smile.process_index(
    df_test.index,
    root=db.root,

)


In [ ]:
X_train.head

<bound method NDFrame.head of                                                   audspec_lengthL1norm_sma_range  \
file            start  end                                                         
wav/03a01Fa.wav 0 days 0 days 00:00:01.898250                           2.935072   
wav/03a01Nc.wav 0 days 0 days 00:00:01.611250                           3.087914   
wav/03a01Wa.wav 0 days 0 days 00:00:01.877812500                        3.673083   
wav/03a02Fc.wav 0 days 0 days 00:00:02.006250                           3.149658   
wav/03a02Nc.wav 0 days 0 days 00:00:01.439812500                        2.437313   
...                                                                          ...   
wav/13b10Fa.wav 0 days 0 days 00:00:02.158562500                        3.092541   
wav/13b10La.wav 0 days 0 days 00:00:02.348437500                        2.381379   
wav/13b10Nc.wav 0 days 0 days 00:00:02.367812500                        2.680195   
wav/13b10Wa.wav 0 days 0 days 00:00:02.2005625

In [ ]:
# labels_encoded = {'anger':0, 'boredom':1, 'disgust':2, 'fear':3, 'happiness':4, 'sadness':5, 'neutral':6}
# df_train['label']=df_train['emotion'].apply(lambda x:labels_encoded[x])
# df_test['label']=df_test['emotion'].apply(lambda x:labels_encoded[x])

y_train=np.array(df_train.label)
y_test=np.array(df_test.label)

In [ ]:
print(y_train.shape)
print(y_test.shape)

(304,)
(231,)


In [ ]:
# Perform a standard scaling / z-transformation on the features (mean=0, std=1)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

**Balancing classes**

In [ ]:
# # balance train classes
sm = SMOTE()
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
encoder = OneHotEncoder(sparse=False)
y_train= y_train.reshape(-1, 1)
print(y_train.shape)
y_train = encoder.fit_transform(y_train)
print(y_train.shape)
y_test= y_test.reshape(-1, 1)
print(y_test.shape)
y_test = encoder.fit_transform(y_test)
print(y_test.shape)

(504, 1)
(504, 7)
(231, 1)
(231, 7)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
print(y_test[0],df_test.label[0])

[0. 0. 0. 0. 1. 0. 0.] 4


In [ ]:
model = models.Sequential([
    layers.Dense(16, activation='relu'),
    # layers.Dense(128, activation='relu'),
    layers.Dense(len(labels_encoded), activation='softmax')  # Output layer with number of emotions
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(str(model.summary()))

In [ ]:
best_weights_file = "nn_weights.h5"
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=10)
mc = ModelCheckpoint(best_weights_file, monitor='val_loss', mode='min', verbose=2,
                         save_best_only=True)
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2,callbacks=[es, mc])

Epoch 1/30
 9/13 [===================>..........] - ETA: 0s - loss: 1.3072 - accuracy: 0.5312
Epoch 1: val_loss improved from inf to 0.20943, saving model to nn_weights.h5
13/13 [==============================] - 2s 64ms/step - loss: 1.1178 - accuracy: 0.6055 - val_loss: 0.2094 - val_accuracy: 0.9604
Epoch 2/30
 9/13 [===================>..........] - ETA: 0s - loss: 0.1542 - accuracy: 0.9618

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.20943 to 0.06794, saving model to nn_weights.h5
13/13 [==============================] - 0s 26ms/step - loss: 0.1389 - accuracy: 0.9628 - val_loss: 0.0679 - val_accuracy: 0.9703
Epoch 3/30
11/13 [========================>.....] - ETA: 0s - loss: 0.0361 - accuracy: 0.9915
Epoch 3: val_loss improved from 0.06794 to 0.03192, saving model to nn_weights.h5
13/13 [==============================] - 0s 28ms/step - loss: 0.0350 - accuracy: 0.9926 - val_loss: 0.0319 - val_accuracy: 0.9901
Epoch 4/30
13/13 [==============================] - ETA: 0s - loss: 0.0154 - accuracy: 0.9975
Epoch 4: val_loss improved from 0.03192 to 0.02567, saving model to nn_weights.h5
13/13 [==============================] - 0s 34ms/step - loss: 0.0154 - accuracy: 0.9975 - val_loss: 0.0257 - val_accuracy: 0.9901
Epoch 5/30
13/13 [==============================] - ETA: 0s - loss: 0.0102 - accuracy: 0.9975
Epoch 5: val_loss improved from 0.02567 to 0.02274, saving model to nn_weights.h5

In [ ]:
# test acc and loss
model.load_weights(best_weights_file) # load the best saved model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
test_metrics = model.evaluate(X_test, y_test, batch_size=32)
print("\n%s: %.2f%%" % ("test " + model.metrics_names[1], test_metrics[1] * 100))
print("%s: %.2f" % ("test " + model.metrics_names[0], test_metrics[0]))
print("test accuracy: " + str(format(test_metrics[1], '.3f')) + "\n")
print("test loss: " + str(format(test_metrics[0], '.3f')) + "\n")
# test acc and loss per class

8/8 [==============================] - 0s 3ms/step - loss: 1.3106 - accuracy: 0.7273

test accuracy: 72.73%
test loss: 1.31
test accuracy: 0.727

test loss: 1.311



Using SVM classifier model

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [ ]:
clf.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [ ]:
y_predict= clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

In [ ]:
accuracy_score(y_test,y_predict)

0.7402597402597403

In [ ]:
confusion_matrix(df_test.emotion,y_predict)

array([[55,  0,  0,  0,  0,  0,  0],
       [ 1, 31,  0,  0,  0,  4,  0],
       [ 3,  3, 14,  1,  4,  1,  0],
       [14,  0,  0, 18,  1,  0,  0],
       [24,  0,  0,  0,  3,  0,  0],
       [ 1,  1,  0,  1,  0, 24,  0],
       [ 0,  1,  0,  0,  0,  0, 26]])